<a href="https://colab.research.google.com/github/martin-steinegger/kraken-protocol/blob/main/Kraken_microbiom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Metagenome analysis using the Kraken software suite - Microbial analysis

This notebook executes the "Microbiome Analysis" workflow described in the manuscript "Metagenome analysis using the Kraken software suite". To execute this notebook click "Runtime" -> "Run all"'. Running this whole notebook takes approx. **30-40 minutes**! To see the code click "Show code" under the section of interest.

In [ ]:
#@title Install Software using Conda
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

!conda create -yqn tutorial -c conda-forge -c bioconda kraken2 krakentools bracken
# in a shell you would run the following, however we want the binaries to be available in this notebook
# source activate tutorial
conda_path = !source activate tutorial && echo $CONDA_PREFIX:$PATH
%set_env PATH=$conda_path

In [ ]:
#@title Download Samples
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/008/SRR13697208/SRR13697208_1.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/008/SRR13697208/SRR13697208_2.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/054/SRR13697154/SRR13697154_1.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/054/SRR13697154/SRR13697154_2.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/036/SRR13697136/SRR13697136_1.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/036/SRR13697136/SRR13697136_2.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/069/SRR13697069/SRR13697069_1.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/069/SRR13697069/SRR13697069_2.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/067/SRR13697067/SRR13697067_1.fastq.gz
!wget -qnc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR136/067/SRR13697067/SRR13697067_2.fastq.gz
for sample in ["SRR13697208", "SRR13697154", "SRR13697136", "SRR13697069", "SRR13697067"]: 
  !gunzip {sample}_1.fastq.gz 
  !gunzip {sample}_2.fastq.gz

In [ ]:
#@title Download Reference Database
!mkdir protocol_db
!wget -qnc https://genome-idx.s3.amazonaws.com/kraken/k2_standard_eupath_20201202.tar.gz
!tar xvfz k2_standard_eupath_20201202.tar.gz -C protocol_db

In [ ]:
#@title Classify Reads using Kraken2
for sample in ["SRR13697208", "SRR13697154", "SRR13697136", "SRR13697069", "SRR13697067"]: 
  !kraken2 --db protocol_db --report {sample}.k2report --report-minimizer-data --minimum-hit-groups 3 {sample}_1.fastq {sample}_2.fastq > {sample}.kraken2

In [ ]:
#@title Estimate Abundance of Species using Bracken
for sample in ["SRR13697208", "SRR13697154", "SRR13697136", "SRR13697069", "SRR13697067"]: 
  !bracken -d protocol_db -i {sample}.k2report -o {sample}.bracken -w {sample}.breport -r 100 -l S -t 10

In [ ]:
#@title Compute Alpha Diversity
for sample in ["SRR13697208", "SRR13697154", "SRR13697136", "SRR13697069", "SRR13697067"]: 
  !alpha_diversity.py -f {sample}.bracken -a BP

In [ ]:
#@title Compute Beta Diversity
!beta_diversity.py -i SRR13697069.bracken SRR13697067.bracken SRR13697136.bracken --type bracken

In [ ]:
#@title Visualizing Taxonomy using Krona
for sample in ["SRR13697208", "SRR13697154", "SRR13697136", "SRR13697069", "SRR13697067"]: 
  !kreport2krona.py -r {sample}.k2report -o {sample}.b.krona.txt --no-intermediate-ranks

In [ ]:
#@title Download SRR13697154.k2report
from google.colab import files
files.download('SRR13697154.k2report') 

##Visualizing Sample using Pavian

In [ ]:
#Visualizing Sample using Pavian
from IPython.display import IFrame
IFrame('https://fbreitwieser.shinyapps.io/pavian/', width='100%', height=600)